In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Для работы с матрицами
from scipy.sparse import csr_matrix

# Матричная факторизация
from implicit import als

# Модель второго уровня
from lightgbm import LGBMClassifier

import os, sys
module_path = os.path.abspath(os.path.join(os.pardir))
if module_path not in sys.path:
    sys.path.append(module_path)

# Написанные нами функции
#from src.metrics import precision_at_k, recall_at_k
#from src.utils import prefilter_items
#from src.recommenders import MainRecommender

In [2]:
data = pd.read_csv('K:/2020/Programming/Data Science/Рекомендательные системы/Урок 3/test/retail_train.csv')
item_features = pd.read_csv('K:/2020/Programming/Data Science/Рекомендательные системы/Урок 3/test/product.csv')
user_features = pd.read_csv('K:/2020/Programming/Data Science/Рекомендательные системы/Урок 3/test/hh_demographic.csv')

# column processing
item_features.columns = [col.lower() for col in item_features.columns]
user_features.columns = [col.lower() for col in user_features.columns]

item_features.rename(columns={'product_id': 'item_id'}, inplace=True)
user_features.rename(columns={'household_key': 'user_id'}, inplace=True)


# Важна схема обучения и валидации!
# -- давние покупки -- | -- 6 недель -- | -- 3 недель -- 
# подобрать размер 2-ого датасета (6 недель) --> learning curve (зависимость метрики recall@k от размера датасета)
val_lvl_1_size_weeks = 6
val_lvl_2_size_weeks = 3

data_train_lvl_1 = data[data['week_no'] < data['week_no'].max() - (val_lvl_1_size_weeks + val_lvl_2_size_weeks)]
data_val_lvl_1 = data[(data['week_no'] >= data['week_no'].max() - (val_lvl_1_size_weeks + val_lvl_2_size_weeks)) &
                      (data['week_no'] < data['week_no'].max() - (val_lvl_2_size_weeks))]

data_train_lvl_2 = data_val_lvl_1.copy()  # Для наглядности. Далее мы добавим изменения, и они будут отличаться
data_val_lvl_2 = data[data['week_no'] >= data['week_no'].max() - val_lvl_2_size_weeks]

data_train_lvl_1.head(2)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.6,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.0,1631,1,0.0,0.0


In [3]:
# recommenders

import pandas as pd
import numpy as np

# Для работы с матрицами
from scipy.sparse import csr_matrix

# Матричная факторизация
from implicit.als import AlternatingLeastSquares
from implicit.nearest_neighbours import ItemItemRecommender  # нужен для одного трюка
from implicit.nearest_neighbours import bm25_weight, tfidf_weight


class MainRecommender:
    """Рекоммендации, которые можно получить из ALS
    Input
    -----
    user_item_matrix: pd.DataFrame
        Матрица взаимодействий user-item
    """

    def __init__(self, data, weighting=True):

        # Топ покупок каждого юзера
        self.top_purchases = data.groupby(['user_id', 'item_id'])['quantity'].count().reset_index()
        self.top_purchases.sort_values('quantity', ascending=False, inplace=True)
        self.top_purchases = self.top_purchases[self.top_purchases['item_id'] != 999999]

        # Топ покупок по всему датасету
        self.overall_top_purchases = data.groupby('item_id')['quantity'].count().reset_index()
        self.overall_top_purchases.sort_values('quantity', ascending=False, inplace=True)
        self.overall_top_purchases = self.overall_top_purchases[self.overall_top_purchases['item_id'] != 999999]
        self.overall_top_purchases = self.overall_top_purchases.item_id.tolist()

        self.user_item_matrix = self._prepare_matrix(data)  # pd.DataFrame
        self.id_to_itemid, self.id_to_userid, \
            self.itemid_to_id, self.userid_to_id = self._prepare_dicts(self.user_item_matrix)

        if weighting:
            self.user_item_matrix = bm25_weight(self.user_item_matrix.T).T

        self.model = self.fit(self.user_item_matrix)
        self.own_recommender = self.fit_own_recommender(self.user_item_matrix)

    @staticmethod
    def _prepare_matrix(data):
        """Готовит user-item матрицу"""
        user_item_matrix = pd.pivot_table(data,
                                          index='user_id', columns='item_id',
                                          values='quantity',  # Можно пробовать другие варианты
                                          aggfunc='count',
                                          fill_value=0
                                          )

        user_item_matrix = user_item_matrix.astype(float)  # необходимый тип матрицы для implicit

        return user_item_matrix

    @staticmethod
    def _prepare_dicts(user_item_matrix):
        """Подготавливает вспомогательные словари"""

        userids = user_item_matrix.index.values
        itemids = user_item_matrix.columns.values

        matrix_userids = np.arange(len(userids))
        matrix_itemids = np.arange(len(itemids))

        id_to_itemid = dict(zip(matrix_itemids, itemids))
        id_to_userid = dict(zip(matrix_userids, userids))

        itemid_to_id = dict(zip(itemids, matrix_itemids))
        userid_to_id = dict(zip(userids, matrix_userids))

        return id_to_itemid, id_to_userid, itemid_to_id, userid_to_id

    @staticmethod
    def fit_own_recommender(user_item_matrix):
        """Обучает модель, которая рекомендует товары, среди товаров, купленных юзером"""

        own_recommender = ItemItemRecommender(K=1, num_threads=4)
        own_recommender.fit(csr_matrix(user_item_matrix).T.tocsr())

        return own_recommender

    @staticmethod
    def fit(user_item_matrix, n_factors=20, regularization=0.001, iterations=15, num_threads=4):
        """Обучает ALS"""

        model = AlternatingLeastSquares(factors=n_factors,
                                        regularization=regularization,
                                        iterations=iterations,
                                        num_threads=num_threads)
        model.fit(csr_matrix(user_item_matrix).T.tocsr())

        return model

    def _update_dict(self, user_id):
        """Если появился новыю user / item, то нужно обновить словари"""

        if user_id not in self.userid_to_id.keys():

            max_id = max(list(self.userid_to_id.values()))
            max_id += 1

            self.userid_to_id.update({user_id: max_id})
            self.id_to_userid.update({max_id: user_id})

    def _get_similar_item(self, item_id):
        """Находит товар, похожий на item_id"""
        recs = self.model.similar_items(self.itemid_to_id[item_id], N=2)  # Товар похож на себя -> рекомендуем 2 товара
        top_rec = recs[1][0]  # И берем второй (не товар из аргумента метода)
        return self.id_to_itemid[top_rec]

    def _extend_with_top_popular(self, recommendations, N=5):
        """Если кол-во рекоммендаций < N, то дополняем их топ-популярными"""

        if len(recommendations) < N:
            recommendations.extend(self.overall_top_purchases[:N])
            recommendations = recommendations[:N]

        return recommendations

    def _get_recommendations(self, user, model, N=5):
        """Рекомендации через стардартные библиотеки implicit"""

        self._update_dict(user_id=user)
        res = [self.id_to_itemid[rec[0]] for rec in model.recommend(userid=self.userid_to_id[user],
                                        user_items=csr_matrix(self.user_item_matrix).tocsr(),
                                        N=N,
                                        filter_already_liked_items=False,
                                        filter_items=[self.itemid_to_id[999999]],
                                        recalculate_user=True)]

        res = self._extend_with_top_popular(res, N=N)

        assert len(res) == N, 'Количество рекомендаций != {}'.format(N)
        return res

    def get_als_recommendations(self, user, N=5):
        """Рекомендации через стардартные библиотеки implicit"""

        self._update_dict(user_id=user)
        return self._get_recommendations(user, model=self.model, N=N)

    def get_own_recommendations(self, user, N=5):
        """Рекомендуем товары среди тех, которые юзер уже купил"""

        self._update_dict(user_id=user)
        return self._get_recommendations(user, model=self.own_recommender, N=N)

    def get_similar_items_recommendation(self, user, N=5):
        """Рекомендуем товары, похожие на топ-N купленных юзером товаров"""

        top_users_purchases = self.top_purchases[self.top_purchases['user_id'] == user].head(N)

        res = top_users_purchases['item_id'].apply(lambda x: self._get_similar_item(x)).tolist()
        res = self._extend_with_top_popular(res, N=N)

        assert len(res) == N, 'Количество рекомендаций != {}'.format(N)
        return res

    def get_similar_users_recommendation(self, user, N=5):
        """Рекомендуем топ-N товаров, среди купленных похожими юзерами"""

        res = []

        # Находим топ-N похожих пользователей
        similar_users = self.model.similar_users(self.userid_to_id[user], N=N+1)
        similar_users = [rec[0] for rec in similar_users]
        similar_users = similar_users[1:]   # удалим юзера из запроса

        for user in similar_users:
            res.extend(self.get_own_recommendations(user, N=1))

        res = self._extend_with_top_popular(res, N=N)

        assert len(res) == N, 'Количество рекомендаций != {}'.format(N)
        return res

In [4]:
# metrics

import numpy as np


def precision(recommended_list, bought_list):
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)

    flags = np.isin(bought_list, recommended_list)

    precision = flags.sum() / len(recommended_list)

    return precision


def precision_at_k(recommended_list, bought_list, k=5):
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    

    bought_list = bought_list  # Тут нет [:k] !!
    
    if k < len(recommended_list):
        recommended_list = recommended_list[:k]

    flags = np.isin(bought_list, recommended_list)

    precision = flags.sum() / len(recommended_list)

    return precision


def money_precision_at_k(recommended_list, bought_list, prices_recommended, k=5):
    # your_code
    # Лучше считать через скалярное произведение, а не цикл

    return precision


def recall(recommended_list, bought_list):
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)

    flags = np.isin(bought_list, recommended_list)

    recall = flags.sum() / len(bought_list)

    return recall


def recall_at_k(recommended_list, bought_list, k=5):

    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)

    if k < len(recommended_list):
        recommended_list = recommended_list[:k]

    flags = np.isin(bought_list, recommended_list)
    recall = flags.sum() / len(bought_list)

    return recall


def money_recall_at_k(recommended_list, bought_list, prices_recommended, prices_bought, k=5):
    # your_code

    return recall

In [5]:
# utils

import pandas as pd
import numpy as np


def prefilter_items(data, take_n_popular=5000, item_features=None):
    # Уберем самые популярные товары (их и так купят)
    popularity = data.groupby('item_id')['user_id'].nunique().reset_index() / data['user_id'].nunique()
    popularity.rename(columns={'user_id': 'share_unique_users'}, inplace=True)

    top_popular = popularity[popularity['share_unique_users'] > 0.2].item_id.tolist()
    data = data[~data['item_id'].isin(top_popular)]

    # Уберем самые НЕ популярные товары (их и так НЕ купят)
    top_notpopular = popularity[popularity['share_unique_users'] < 0.02].item_id.tolist()
    data = data[~data['item_id'].isin(top_notpopular)]

    # Уберем товары, которые не продавались за последние 12 месяцев

    # Уберем не интересные для рекоммендаций категории (department)
    if item_features is not None:
        department_size = pd.DataFrame(item_features.\
                                        groupby('department')['item_id'].nunique().\
                                        sort_values(ascending=False)).reset_index()

        department_size.columns = ['department', 'n_items']
        rare_departments = department_size[department_size['n_items'] < 150].department.tolist()
        items_in_rare_departments = item_features[item_features['department'].isin(rare_departments)].item_id.unique().tolist()

        data = data[~data['item_id'].isin(items_in_rare_departments)]


    # Уберем слишком дешевые товары (на них не заработаем). 1 покупка из рассылок стоит 60 руб.
    data['price'] = data['sales_value'] / (np.maximum(data['quantity'], 1))
    data = data[data['price'] > 2]

    # Уберем слишком дорогие товарыs
    data = data[data['price'] < 50]

    # Возбмем топ по популярности
    popularity = data.groupby('item_id')['quantity'].sum().reset_index()
    popularity.rename(columns={'quantity': 'n_sold'}, inplace=True)

    top = popularity.sort_values('n_sold', ascending=False).head(take_n_popular).item_id.tolist()
    
    # Заведем фиктивный item_id (если юзер покупал товары из топ-5000, то он "купил" такой товар)
    data.loc[~data['item_id'].isin(top), 'item_id'] = 999999
    
    # ...

    return data


def postfilter_items(user_id, recommednations):
    pass

In [6]:
n_items_before = data_train_lvl_1['item_id'].nunique()

data_train_lvl_1 = prefilter_items(data_train_lvl_1, item_features=item_features, take_n_popular=5000)

n_items_after = data_train_lvl_1['item_id'].nunique()
print('Decreased # items from {} to {}'.format(n_items_before, n_items_after))

Decreased # items from 83685 to 5001


In [7]:
recommender = MainRecommender(data_train_lvl_1)

  0%|          | 0/15 [00:00<?, ?it/s]

  0%|          | 0/5001 [00:00<?, ?it/s]

In [8]:
recommender

In [9]:
recommender.get_als_recommendations(2375, N=5)

[1106523, 899624, 871756, 1044078, 883932]

In [10]:
recommender.get_own_recommendations(2375, N=5)

[948640, 918046, 847962, 907099, 873980]

In [11]:
recommender.get_similar_items_recommendation(2375, N=5)

[1046545, 7147915, 1044078, 907099, 846764]

In [12]:
recommender.get_similar_users_recommendation(2375, N=5)

[820612, 5574377, 1012801, 1101502, 9553048]

In [13]:
result_lvl_1 = data_val_lvl_1.groupby('user_id')['item_id'].unique().reset_index()
result_lvl_1.columns=['user_id', 'actual']
result_lvl_1.head(2)

,user_id,actual
0,1,"[853529, 865456, 867607, 872137, 874905, 87524..."
1,2,"[15830248, 838136, 839656, 861272, 866211, 870..."


In [14]:
n_items_before = data_val_lvl_1['item_id'].nunique()

data_val_lvl_1 = prefilter_items(data_val_lvl_1, item_features=item_features, take_n_popular=5000)

n_items_after = data_val_lvl_1['item_id'].nunique()
print('Decreased # items from {} to {}'.format(n_items_before, n_items_after))

Decreased # items from 27649 to 5001


In [15]:
# Проверим метрики recall_at_k по моделям als, own и similar_items - наибольшая у own

In [16]:
recommender = MainRecommender(data_val_lvl_1)

  0%|          | 0/15 [00:00<?, ?it/s]

  0%|          | 0/5001 [00:00<?, ?it/s]

In [17]:
result_lvl_1 = data_val_lvl_1.groupby('user_id')['item_id'].unique().reset_index()
result_lvl_1.columns=['user_id', 'actual']
result_lvl_1.head(2)

,user_id,actual
0,1,"[865456, 999999, 878285, 922281, 940947, 98389..."
1,2,"[999999, 839656, 866211, 970481, 980666, 10114..."


In [18]:
result_lvl_1['als'] = result_lvl_1['user_id'].apply(lambda x: recommender.get_als_recommendations(x, N=50))

In [19]:
result_lvl_1.head(2)

,user_id,actual,als
0,1,"[865456, 999999, 878285, 922281, 940947, 98389...","[8293439, 9297615, 1069103, 909497, 963542, 87..."
1,2,"[999999, 839656, 866211, 970481, 980666, 10114...","[1111116, 1103898, 970481, 9419487, 17215077, ..."


In [20]:
result_lvl_1.apply(lambda row: recall_at_k(row['als'], row['actual']), axis=1).mean()

0.16691575940381778

In [21]:
result_lvl_1['own'] = result_lvl_1['user_id'].apply(lambda x: recommender.get_own_recommendations(x, N=50))
result_lvl_1.head(2)

,user_id,actual,als,own
0,1,"[865456, 999999, 878285, 922281, 940947, 98389...","[8293439, 9297615, 1069103, 909497, 963542, 87...","[8293439, 1069103, 8293343, 9297615, 983897, 9..."
1,2,"[999999, 839656, 866211, 970481, 980666, 10114...","[1111116, 1103898, 970481, 9419487, 17215077, ...","[1111116, 7407562, 970481, 8357614, 997987, 11..."


In [22]:
result_lvl_1.apply(lambda row: recall_at_k(row['own'], row['actual']), axis=1).mean()

0.36303367135278547

In [23]:
result_lvl_1['similar_items'] = result_lvl_1['user_id'].apply(lambda x: recommender.get_similar_items_recommendation(x, N=50))
result_lvl_1.head(2)

,user_id,actual,als,own,similar_items
0,1,"[865456, 999999, 878285, 922281, 940947, 98389...","[8293439, 9297615, 1069103, 909497, 963542, 87...","[8293439, 1069103, 8293343, 9297615, 983897, 9...","[13002975, 5569230, 5563693, 899624, 1104343, ..."
1,2,"[999999, 839656, 866211, 970481, 980666, 10114...","[1111116, 1103898, 970481, 9419487, 17215077, ...","[1111116, 7407562, 970481, 8357614, 997987, 11...","[844282, 844179, 12301100, 848029, 1127831, 99..."


In [24]:
result_lvl_1.apply(lambda row: recall_at_k(row['similar_items'], row['actual']), axis=1).mean()

0.04065637693050815

In [25]:
# построим модель 2-го уровня на данных data_val_lvl_2

In [26]:
item_features.head(2)

,item_id,manufacturer,department,brand,commodity_desc,sub_commodity_desc,curr_size_of_product
0,25671,2,GROCERY,National,FRZN ICE,ICE - CRUSHED/CUBED,22 LB
1,26081,2,MISC. TRANS.,National,NO COMMODITY DESCRIPTION,NO SUBCOMMODITY DESCRIPTION,


In [27]:
user_features.head(2)

,age_desc,marital_status_code,income_desc,homeowner_desc,hh_comp_desc,household_size_desc,kid_category_desc,user_id
0,65+,A,35-49K,Homeowner,2 Adults No Kids,2,None/Unknown,1
1,45-54,A,50-74K,Homeowner,2 Adults No Kids,2,None/Unknown,7


In [28]:
n_items_before = data_val_lvl_2['item_id'].nunique()

data_val_lvl_2 = prefilter_items(data_val_lvl_2, item_features=item_features, take_n_popular=5000)

n_items_after = data_val_lvl_2['item_id'].nunique()
print('Decreased # items from {} to {}'.format(n_items_before, n_items_after))

Decreased # items from 24329 to 5001


In [29]:
recommender = MainRecommender(data_val_lvl_2)

  0%|          | 0/15 [00:00<?, ?it/s]

  0%|          | 0/5001 [00:00<?, ?it/s]

In [30]:
recommender

In [31]:
users_val_2 = pd.DataFrame(data_val_lvl_2['user_id'].unique())
users_val_2.columns = ['user_id']

# Пока только warm start
train_users = data_val_lvl_2['user_id'].unique()
users_val_2 = users_val_2[users_val_2['user_id'].isin(train_users)]

users_val_2['candidates'] = users_val_2['user_id'].apply(lambda x: recommender.get_own_recommendations(x, N=50))

In [32]:
users_val_2.head(2)

,user_id,candidates
0,338,"[959737, 901827, 13002975, 5586918, 965637, 92..."
1,2120,"[2049730, 1029743, 1106523, 5569230, 1070820, ..."


In [33]:
s = users_val_2.apply(lambda x: pd.Series(x['candidates']), axis=1).stack().reset_index(level=1, drop=True)
s.name = 'item_id'

users_val_2 = users_val_2.drop('candidates', axis=1).join(s)
# flag - столбец для рекомендованных моделью товаров (из логики все что рекомендовала модель = считает что пользователь купит такой товар, те prediction = 1)
users_val_2['flag'] = 1

users_val_2.head(4)

,user_id,item_id,flag
0,338,959737,1
0,338,901827,1
0,338,13002975,1
0,338,5586918,1


In [34]:
users_val_2.shape[0]

99500

In [35]:
users_val_2['user_id'].nunique()

1990

In [36]:
targets_val_2 = data_val_lvl_2[['user_id', 'item_id']].copy()
targets_val_2['target'] = 1  # тут только покупки (те фактические данные - для расчета recal и precision)

targets_val_2 = users_val_2.merge(targets_val_2, on=['user_id', 'item_id'], how='left')

targets_val_2['target'].fillna(0, inplace= True)
targets_val_2.drop('flag', axis=1, inplace=True)

In [37]:
targets_val_2.head(2)

,user_id,item_id,target
0,338,959737,1.0
1,338,959737,1.0


In [38]:
targets_val_2['target'].mean()

0.33048755698812327

In [39]:
data_val_lvl_2.head(2)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc,price
2277419,338,41260573635,636,7441679,1,3.69,369,0.0,112,92,0.0,0.0,3.69
2277420,338,41260573635,636,7442317,1,2.69,369,0.0,112,92,0.0,0.0,2.69


In [40]:
user_features_2 = data_val_lvl_2

In [41]:
# добавим фичу средний чек

In [42]:
user_features_2['check'] = user_features_2['sales_value']

In [43]:
av_check = user_features_2.groupby('user_id')['check'].mean().reset_index()
av_check

,user_id,check
0,1,3.433846
1,3,17.070000
2,6,4.828636
3,7,4.220185
4,8,3.981176
...,...,...
1985,2495,2.490000
1986,2497,5.156421
1987,2498,3.529286
1988,2499,3.925333


In [44]:
# добавим фичу цены продукта

In [45]:
price = user_features_2.groupby('item_id')['price'].sum().reset_index()
price

,item_id,price
0,819210,4.380000
1,819308,41.909167
2,819423,35.960000
3,819487,19.140000
4,819840,14.830000
...,...,...
4996,17903379,5.850000
4997,17903423,3.990000
4998,17959083,28.760000
4999,18000012,14.970000


In [46]:
# добавим фичу средней цены продукта

In [47]:
av_price = user_features_2.groupby('item_id')['price'].sum()/user_features_2.groupby('item_id')['price'].mean()
av_price

item_id
819210       2.0
819308      15.0
819423       4.0
819487       6.0
819840       5.0
            ... 
17903379     2.0
17903423     1.0
17959083     4.0
18000012     3.0
18024556     3.0
Name: price, Length: 5001, dtype: float64

In [48]:
# добавим фичу средней скидки

In [49]:
av_disc = user_features_2.groupby('user_id')['retail_disc'].mean().reset_index()
av_disc

,user_id,retail_disc
0,1,-0.341538
1,3,-6.610000
2,6,-0.416364
3,7,-0.534074
4,8,-0.635000
...,...,...
1985,2495,0.000000
1986,2497,-0.882632
1987,2498,-0.204643
1988,2499,-1.078667


In [50]:
# добавим фичу средней стоимости покупки в неделю

In [51]:
av_sales_week = user_features_2.groupby('item_id')['sales_value'].sum()/user_features_2.groupby('item_id')['week_no'].sum()
av_sales_week

item_id
819210      0.046105
819308      0.050441
819423      0.095132
819487      0.034057
819840      0.031688
              ...   
17903379    0.046096
17903423    0.168000
17959083    0.076693
18000012    0.070035
18024556    0.107053
Length: 5001, dtype: float64

In [52]:
#user_features_2 = user_features_2.merge(item_features, on='item_id', how='left')
#user_features_2 = user_features_2.merge(user_features, on='user_id', how='left')
#user_features_2 = user_features_2.merge(t2, on='user_id', how='left')
#user_features_2 = user_features_2.merge(t3, on='item_id', how='left')
#user_features_2 = user_features_2.merge(t4, on='item_id', how='left')
#user_features_2 = user_features_2.merge(t5, on='user_id', how='left')
#user_features_2 = user_features_2.merge(t6, on='item_id', how='left')
#user_features_2 = user_features_2.merge(t7, on='item_id', how='left')

In [53]:
targets_val_2 = data_val_lvl_2[['user_id', 'item_id']].copy()
targets_val_2['target'] = 1  # тут только покупки (те фактические данные - для расчета recal и precision)

targets_val_2 = users_val_2.merge(targets_val_2, on=['user_id', 'item_id'], how='left')

targets_val_2['target'].fillna(0, inplace= True)
targets_val_2.drop('flag', axis=1, inplace=True)

In [54]:
targets_val_2 = targets_val_2.merge(item_features, on='item_id', how='left')
targets_val_2 = targets_val_2.merge(user_features, on='user_id', how='left')
targets_val_2 = targets_val_2.merge(av_check, on='user_id', how='left')
targets_val_2 = targets_val_2.merge(price, on='item_id', how='left')
targets_val_2 = targets_val_2.merge(av_price, on='item_id', how='left')
targets_val_2 = targets_val_2.merge(av_disc, on='user_id', how='left')
#targets_val_2 = targets_val_2.merge(av_sales_week, on='item_id', how='left')
#targets_val_2 = targets_val_2.merge(t72, on='item_id', how='left')


In [55]:
targets_val_2.head(2)

,user_id,item_id,target,manufacturer,department,brand,commodity_desc,sub_commodity_desc,curr_size_of_product,age_desc,marital_status_code,income_desc,homeowner_desc,hh_comp_desc,household_size_desc,kid_category_desc,check,price_x,price_y,retail_disc
0,338,959737,1.0,2343,GROCERY,National,BEERS/ALES,BEERALEMALT LIQUORS,12 OZ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7.3762,41.97,3.0,-1.1222
1,338,959737,1.0,2343,GROCERY,National,BEERS/ALES,BEERALEMALT LIQUORS,12 OZ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7.3762,41.97,3.0,-1.1222


In [56]:
cat_feats = targets_val_2.columns[2:].tolist()
#X_train[cat_feats] = X_train[cat_feats].astype('category')

cat_feats

['target',
 'manufacturer',
 'department',
 'brand',
 'commodity_desc',
 'sub_commodity_desc',
 'curr_size_of_product',
 'age_desc',
 'marital_status_code',
 'income_desc',
 'homeowner_desc',
 'hh_comp_desc',
 'household_size_desc',
 'kid_category_desc',
 'check',
 'price_x',
 'price_y',
 'retail_disc']

In [57]:
targets_val_2 = targets_val_2.reindex(columns=['user_id',
                                               'item_id',
                                               'target',
                                               'check',
                                               'price_x',
                                               'price_y',
                                               'retail_disc', 
                                               'manufacturer',
                                               'department',
                                               'brand',
                                               'commodity_desc',
                                               'sub_commodity_desc',
                                               'curr_size_of_product',
                                               'age_desc',
                                               'marital_status_code',
                                               'income_desc',
                                               'homeowner_desc',
                                               'hh_comp_desc',
                                               'household_size_desc',
                                               'kid_category_desc'])

In [58]:
targets_val_2

,user_id,item_id,target,check,price_x,price_y,retail_disc,manufacturer,department,brand,commodity_desc,sub_commodity_desc,curr_size_of_product,age_desc,marital_status_code,income_desc,homeowner_desc,hh_comp_desc,household_size_desc,kid_category_desc
0,338,959737,1.0,7.376200,41.970000,3.0,-1.1222,2343,GROCERY,National,BEERS/ALES,BEERALEMALT LIQUORS,12 OZ,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,338,959737,1.0,7.376200,41.970000,3.0,-1.1222,2343,GROCERY,National,BEERS/ALES,BEERALEMALT LIQUORS,12 OZ,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,338,901827,1.0,7.376200,10.725000,3.0,-1.1222,69,DRUG GM,Private,ANALGESICS,ADULT ANALGESICS,,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,338,13002975,1.0,7.376200,26.766667,7.0,-1.1222,2843,MEAT,National,BEEF,RIBS,,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,338,13002975,1.0,7.376200,26.766667,7.0,-1.1222,2843,MEAT,National,BEEF,RIBS,,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
103306,832,908318,0.0,4.923333,167.440000,56.0,0.0000,5124,PRODUCE,National,POTATOES,POTATOES OTHER,4 CT,NaN,NaN,NaN,NaN,NaN,NaN,NaN
103307,832,865456,0.0,4.923333,223.440000,56.0,0.0000,317,GROCERY,National,CHEESE,LOAF CHEESE,L 2 LB,NaN,NaN,NaN,NaN,NaN,NaN,NaN
103308,832,5585510,0.0,4.923333,221.040000,56.0,0.0000,69,PRODUCE,Private,TOMATOES,TOMATOES VINE RIPE PKG,4 CT,NaN,NaN,NaN,NaN,NaN,NaN,NaN
103309,832,1026118,0.0,4.923333,208.610000,55.0,0.0000,2,PRODUCE,National,TOMATOES,ROMA TOMATOES (BULK/PKG),25 LB,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [59]:
targets_val_2['target'].mean()

0.33048755698812327

In [60]:
X_val = targets_val_2.drop('target', axis=1)
y_val = targets_val_2[['target']]

In [61]:
cat_feats = X_val.columns[6:].tolist()
X_val[cat_feats] = X_val[cat_feats].astype('category')

cat_feats

['manufacturer',
 'department',
 'brand',
 'commodity_desc',
 'sub_commodity_desc',
 'curr_size_of_product',
 'age_desc',
 'marital_status_code',
 'income_desc',
 'homeowner_desc',
 'hh_comp_desc',
 'household_size_desc',
 'kid_category_desc']

In [62]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Для работы с матрицами
from scipy.sparse import csr_matrix

# Матричная факторизация
#from implicit import als

# Модель второго уровня
from lightgbm import LGBMClassifier

import os, sys
module_path = os.path.abspath(os.path.join(os.pardir))
if module_path not in sys.path:
    sys.path.append(module_path)

# Написанные нами функции
#from src.metrics import precision_at_k, recall_at_k
#from src.utils import prefilter_items
#from src.recommenders import MainRecommender

In [68]:
lgb = LGBMClassifier(n_jobs=-1,
    max_depth=6,
    subsample=1,
    n_estimators=100,
    learning_rate=0.1,
    colsample_bytree=1,
    objective='binary',
    boosting_type='gbdt')

lgb.fit(X_val, y_val)

#train_preds = lgb.predict(X_val)

val_preds = lgb.predict(X_val)

C:\Users\user\anaconda3\lib\site-packages\sklearn\preprocessing\_label.py:98: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\user\anaconda3\lib\site-packages\sklearn\preprocessing\_label.py:133: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [69]:
len(X_val)

103311

In [70]:
len(val_preds)

103311

In [71]:
val_preds.mean()

0.27665011470220985

In [72]:
X_val['preds'] = val_preds

In [73]:
X_val.head()

,user_id,item_id,check,price_x,price_y,retail_disc,manufacturer,department,brand,commodity_desc,sub_commodity_desc,curr_size_of_product,age_desc,marital_status_code,income_desc,homeowner_desc,hh_comp_desc,household_size_desc,kid_category_desc,preds
0,338,959737,7.3762,41.970000,3.0,-1.1222,2343,GROCERY,National,BEERS/ALES,BEERALEMALT LIQUORS,12 OZ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
1,338,959737,7.3762,41.970000,3.0,-1.1222,2343,GROCERY,National,BEERS/ALES,BEERALEMALT LIQUORS,12 OZ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
2,338,901827,7.3762,10.725000,3.0,-1.1222,69,DRUG GM,Private,ANALGESICS,ADULT ANALGESICS,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
3,338,13002975,7.3762,26.766667,7.0,-1.1222,2843,MEAT,National,BEEF,RIBS,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
4,338,13002975,7.3762,26.766667,7.0,-1.1222,2843,MEAT,National,BEEF,RIBS,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0


In [74]:
X_val.preds.value_counts()

0.0    74730
1.0    28581
Name: preds, dtype: int64

In [75]:
X_val['y_true'] = y_val

In [76]:
X_val['y_true'].value_counts()

0.0    69168
1.0    34143
Name: y_true, dtype: int64

In [77]:
X_val[X_val.preds == 1]['user_id'].nunique()

1893

In [78]:
X_val['user_id'].nunique()

1990

In [79]:
val_data_result = X_val[X_val['y_true']>0].groupby('user_id')['item_id'].agg(list).reset_index()

In [80]:
val_data_result.head(2)

,user_id,item_id
0,1,"[856942, 856942, 9677939, 6534544, 15971874, 9..."
1,3,[958154]


In [81]:
preds = X_val[X_val['preds']>0].groupby('user_id')['item_id'].agg(list).reset_index()

In [82]:
preds.head(2)

,user_id,item_id
0,1,"[856942, 856942, 9677939, 6534544, 15971874, 9..."
1,3,[958154]


In [83]:
val_data_result.rename(columns={'item_id':'actual'}, inplace = True)

In [84]:
preds.head(2)

,user_id,item_id
0,1,"[856942, 856942, 9677939, 6534544, 15971874, 9..."
1,3,[958154]


In [85]:
val_data_result = val_data_result.merge(preds, how='left', on='user_id')

In [86]:
val_data_result.head()

,user_id,actual,item_id
0,1,"[856942, 856942, 9677939, 6534544, 15971874, 9...","[856942, 856942, 9677939, 6534544, 15971874, 9..."
1,3,[958154],[958154]
2,6,"[14106553, 12731467, 1020683, 15863831, 741017...","[14106553, 12731467, 1020683, 15863831, 741017..."
3,7,"[13417587, 861246, 12731506, 10285022, 1045618...","[13417587, 861246, 12731506, 10285022, 1045618..."
4,8,"[16223130, 16223130, 1019751, 7168968, 1086196...","[16223130, 16223130, 1019751, 7168968, 1086196..."


In [85]:
val_data_result.rename(columns={'item_id':'lightGBM'}, inplace = True)

In [86]:
val_data_result.head()

,user_id,actual,lightGBM
0,1,"[856942, 856942, 9677939, 6534544, 15971874, 9...","[856942, 856942, 9677939, 6534544, 15971874, 9..."
1,3,[958154],[958154]
2,6,"[14106553, 12731467, 1020683, 15863831, 741017...","[14106553, 12731467, 1020683, 15863831, 741017..."
3,7,"[13417587, 861246, 12731506, 10285022, 1045618...","[13417587, 861246, 12731506, 10285022, 1045618..."
4,8,"[16223130, 16223130, 1019751, 7168968, 1086196...","[16223130, 16223130, 1019751, 7168968, 1086196..."


In [87]:
def precision_at_k(recommended_list, bought_list, k=5):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    
    bought_list = bought_list  # Тут нет [:k] !!
    try:
        recommended_list = recommended_list[:k]
    except:
        recommended_list = []
    
    flags = np.isin(bought_list, recommended_list)
    
    precision = flags.sum() / len(recommended_list)
    
    
    return precision

In [88]:
val_data_result.apply(lambda row: precision_at_k(row['lightGBM'], row['actual']), axis=1).mean()

C:\Users\user\AppData\Local\Temp\ipykernel_15096\1246819944.py:14: RuntimeWarning: invalid value encountered in long_scalars
  precision = flags.sum() / len(recommended_list)


1.0156002115282918